# Assignment 5

In this assignment, your task is to scrape data from the given [link](https://resultadoshistorico.onpe.gob.pe/EG2021/). You'll need to extract all votes for departments, provinces, and districts. Follow these steps:

1. Click on 'Resumen General'.
2. Select 'Presidencial', and then choose 'Peru'.
3. Scrape the data and generate a final table with the following columns: department, province, district, political_organization, total_votes.

Your code must execute successfully; any errors will result in a zero grade. Ensure you test your code thoroughly.

In [111]:
!pip install selenium
!pip install webdriver-manager
!pip install lxml
!pip install lxmunidecodel

#We install the libraries

ERROR: Could not find a version that satisfies the requirement lxmunidecodel (from versions: none)
ERROR: No matching distribution found for lxmunidecodel


In [112]:
# this library is to manipulate browser
from selenium import webdriver

# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import unidecode
from selenium.webdriver.support.ui import Select  # Import Select class

In [113]:
pwd

'C:\\Users\\joeva\\Documentos\\GitHub\\MediaLab_Summer_Python\\assignments\\assignment5'

In [114]:
# We copy the PATH of the driver
service = Service(executable_path=r"C:\Users\joeva\Documentos\GitHub\MediaLab_Summer_Python\Lecture_7\chromedriver-win64\chromedriver.exe")

driver = webdriver.Chrome(service=service)

url = 'https://resultadoshistorico.onpe.gob.pe/EG2021/'

driver.get(url)

#We maximize the browser and set the Browser Zoom Level to 100 percent

driver.maximize_window()

driver.execute_script("document.body.style.zoom='100%'")

 1. We select the resumen general

In [116]:
resumen_general = driver.find_element( By.XPATH  , '/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[1]/div/div/a/div[2]/div')
resumen_general.click()

2. We select presidencial result

In [117]:
presidencial = driver.find_element( By.XPATH  , '/html/body/onpe-root/onpe-layout-container/onpe-menu/div/nav/div/div/div[2]/div/div[2]/a')
presidencial.click()

Then, we select Perú 

In [119]:
peru = driver.find_element( By.XPATH  , '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select/option[2]')
peru.click()

3. Scrape the data and generate a final table with the following columns: department, province, district, political_organization, total_votes.

In [121]:
# Store all_tables
all_tables = {}

# we loop over all departments
for dpt_idx in range( 26 ):
    
    # Get again all departments since HTML is refreshing
    # all elements
    # Click on one specific department
    dpt = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )
    department = dpt.options[ dpt_idx ]
    
    # Get departmant name
    dpt_name = department.text
    
    # We select a different department name
    if dpt_name != "--TODOS--" :
        
        # click on department
        department.click()
        
        # Get all elements of province
        prov = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
        num_prov_options = len( prov.options )
        
        for prov_idx in range( num_prov_options ):
            
            # Get again all districts since HTML is refreshing
            # all elements
            prov = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
            province = prov.options[ prov_idx ]
                
            # Get province name
            prov_name = province.text
            
            if prov_name != "--TODOS--" :
                
                # click on province
                province.click()
                
                # Get all elements from district
                dist = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                num_dist_options = len( dist.options )
                
                for dist_idx in range( num_dist_options ):
                    
                    # Get again all districts since HTML is refreshing
                    # all elements
                    dist = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                    district = dist.options[ dist_idx ]
                    
                    # Get district name
                    dist_name = district.text
                    
                    if dist_name != "-- SELECCIONE --" :
                        
                        # click on district
                        district.click()
                        
                        # Get UBIGEO
                        ubigeo = driver.current_url.split("/")[ -1 ]
                        
                        ## Gbet table of presidential votes
                        # Get html at this point
                        table_path = driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div" )
                        table_html = table_path.get_attribute( 'innerHTML' )
                        # Read the table using pandas
                        table = pd.read_html( table_html )
                        
                        # Cleaning tables
                        row_new_columns = table[ 0 ].iloc[ 0 , 2: ]
                        clean_columns = row_new_columns \
                                              .str.replace( " ", "_") \
                                              .str.lower().str.replace( "%", "share_") \
                                              .apply( lambda x : unidecode.unidecode( x ) ) \
                                              .tolist()
                        
                        # Selecting specific columns
                        table_clean = table[0].iloc[ 1:, 2: ].copy()
                        
                        # rename columns
                        table_clean.columns = clean_columns
                        
                        # New values to columns 
                        table_clean[ 'department' ] = dpt_name
                        table_clean[ 'province' ]   = prov_name
                        table_clean[ 'district' ]   = dist_name
                    
                        
                        # store tables
                        all_tables[ ubigeo ] = table_clean

In [122]:
final_data = pd.concat( all_tables.values() ).reset_index( drop = True )

In [123]:
final_data

,organizaciones_politicas,total,share_validos,share_emitidos,department,province,district
0,PARTIDO NACIONALISTA PERUANO,356,9.327%,6.389%,AMAZONAS,BAGUA,ARAMANGO
1,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",13,0.341%,0.233%,AMAZONAS,BAGUA,ARAMANGO
2,PARTIDO MORADO,41,1.074%,0.736%,AMAZONAS,BAGUA,ARAMANGO
3,PERU PATRIA SEGURA,4,0.105%,0.072%,AMAZONAS,BAGUA,ARAMANGO
4,VICTORIA NACIONAL,151,3.956%,2.710%,AMAZONAS,BAGUA,ARAMANGO
...,...,...,...,...,...,...,...
41223,ALIANZA PARA EL PROGRESO,292,35.394%,20.434%,UCAYALI,PURUS,PURUS
41224,TOTAL DE VOTOS VÁLIDOS,825,100.000%,57.733%,UCAYALI,PURUS,PURUS
41225,VOTOS EN BLANCO,422,NaN,29.531%,UCAYALI,PURUS,PURUS
41226,VOTOS NULOS,182,NaN,12.736%,UCAYALI,PURUS,PURUS
